Getis Ord Gi* Example¶
Getis and Ord's Gi and Gi* statistics are popular statistical approaches for finding statistically significant hot and cold spots across space. It compares the value of some numerical variable of a spatial record with those of the neighboring records. The nature of these neighborhoods is controlled by the user.

Use the Gi* statistic on the Overture places data to identify regions of high and lower "density".

In [1]:
region = "POLYGON ((-122.380829 47.870302, -122.048492 47.759637, -121.982574 47.531111, -122.408295 47.50978, -122.44812 47.668162, -122.380829 47.870302))"
neighbor_search_radius_degrees = .01
h3_zoom_level = 8

Specifies the search radius for the neighborhood area in geographic degrees. In this case, it is 0.01 degrees, which corresponds to about 1.1 km on the Earth's surface (approximately).

This radius is used in neighborhood search algorithms in the context of the H3 grid to find cells (hexagons) near a specified point.

H3 is a system for dividing the Earth's surface into hexagonal cells, and h3_zoom_level determines the accuracy of these cells. The H3 zoom level in this case is 8, which means that the hexagonal cells will be relatively small (about 150 meters on a side).

The higher the zoom level, the smaller the H3 cells, allowing for more detailed mapping.

region: Specifies a geographic area in the form of a polygon that will be used in geolocation algorithms.

neighbor_search_radius_degrees: Sets the radius in which neighboring cells will be searched for in degrees.

h3_zoom_level: Specifies the level of detail for dividing the Earth's surface into hexagonal cells in the H3 system.

In [3]:
from sedona.spark import *

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Filtering and Aggregation
Assign an H3 cell to each record and filter down to only the region of interest. Aggregate the places data by the cell idenitier and find the number of places in each cell.

In [19]:
import pyspark.sql.functions as f
places_df = (
    sedona.table("wherobots_open_data.overture_2024_07_22.places_place")
        .select(f.col("geometry"), f.col("categories"))
        .withColumn("h3Cell", ST_H3CellIDs(f.col("geometry"), h3_zoom_level, False)[0])
)

if region is not None:
    places_df = places_df.filter(ST_Intersects(ST_GeomFromText(f.lit(region)), f.col("geometry"))).repartition(100)


.withColumn("h3Cell", ST_H3CellIDs(f.col("geometry"), h3_zoom_level, False)[0]) – creates a new column h3Cell that contains H3 cell IDs (spherical grid system) for geometry locations. The ST_H3CellIDs function converts geometry to H3 cell IDs at the specified zoom level h3_zoom_level. This function returns a list of IDs, and [0] means that we select the first item from this list.

.repartition(100) – ponownie partycjonuje dane w 100 partycji, co może być przydatne, aby zoptymalizować operacje obliczeniowe na dużych zbiorach danych. Pomaga to w równomiernym rozdzieleniu danych na dostępne węzły w klastrze Spark.

In [20]:
SedonaKepler.create_map(places_df, "places")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'places': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [22]:
places_df.show(3)

+--------------------+--------------------+------------------+
|            geometry|          categories|            h3Cell|
+--------------------+--------------------+------------------+
|POINT (-122.32119...|{mexican_restaura...|613207891937067007|
|POINT (-122.31716...|{acupuncture, [al...|613207891398098943|
|POINT (-122.33204...|{business_law, [e...|613207891941261311|
+--------------------+--------------------+------------------+
only showing top 3 rows



H3 identifiers are a type of spatial index that is used to represent spherical surfaces as cells on a hexagonal grid. H3 (Hexagonal Hierarchical Spatial Index) is a spatial indexing system developed by Uber that is used to map geographic data onto spherical space using hexagonal cells.

In [25]:
hexes_df = (
    places_df
        .groupBy(f.col("h3Cell"))
        .agg(f.count("*").alias("num_places")) # how many places in this cell
        .withColumn("geometry", ST_H3ToGeom(f.array(f.col("h3Cell")))[0])
)

In [27]:
hexes_df.show(3)

+------------------+----------+--------------------+
|            h3Cell|num_places|            geometry|
+------------------+----------+--------------------+
|613207897777635327|         8|POLYGON ((-122.10...|
|613207894489300991|         9|POLYGON ((-122.25...|
|613207893124055039|        12|POLYGON ((-122.39...|
+------------------+----------+--------------------+
only showing top 3 rows



This code performs the following operations on the data:

Grouping data (groupBy(f.col("h3Cell"))):

We start by grouping the rows in places_df by the h3Cell column, which contains the H3 cell identifiers.

Each row in places_df represents a place, and the h3Cell column indicates which H3 cell it belongs to.

Aggregation (agg(f.count("*").alias("num_places"))):

For each group (i.e. for each H3 cell), an aggregation is performed that counts how many rows (i.e. places) are in that H3 cell.

f.count("*") counts all the places in a given H3 cell.

alias("num_places") renames this new column to num_places, which means the number of places in a given H3 cell.

Adding a column with geometry (withColumn("geometry", ST_H3ToGeom(f.array(f.col("h3Cell")))[0])):

A new column geometry is added to hexes_df. This column contains the geometry corresponding to each H3 cell.

The ST_H3ToGeom(f.array(f.col("h3Cell"))) function converts H3 (h3Cell) IDs to their corresponding geometries.

The f.array(f.col("h3Cell")) function creates an array with the H3 ID, because the ST_H3ToGeom function expects an array.

[0] means that from this array we select the first (and only) geometry for a given H3 cell. As a result, we have a geometry corresponding to each H3 cell, which is useful when we want to visualize these cells on a map.

Summary of the action:
Grouping data by H3 cells.

Number of places in each H3 cell (num_places column).

Creating a geometry for each H3 cell, represented in the geometry column.

After this operation you get a DataFrame (hexes_df) that contains:

The h3Cell column (H3 cell identifier),

The num_places column (the number of places in this cell),

The geometry column (the geometry corresponding to the H3 cell).

This allows you to easily analyze how many places there are in a given H3 cell, and also use the H3 geometries for further spatial analysis or visualization.

In [28]:
SedonaKepler.create_map(hexes_df, "hexes")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'hexes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

Sanity Check Variable
Make sure that have a good distribution of values in variable. Specifically ensuring that cells are not too small which would be indicated by the places counts all being very low. Generate deciles here to make sure that there is some good range of these values. An extreme negative example would be if these values were all zero and one.

In [29]:
hexes_df.select(f.percentile_approx("num_places", [x / 10.0 for x in range(11)])).collect()[0][0]

[1, 2, 3, 5, 8, 11, 18, 32, 63, 118, 2006]

This means that:

0% of the spaces have less than 1 space.

10% of the H3 cells have less than 2 spaces.

20% of the cells have less than 3 spaces, and so on, up to 100% of the cells that have less than 2006 spaces.
Median: 8 means that half of the H3 cells contain 8 or fewer sites, and the other half contain more than 8 sites.

In other words, the median represents the typical number of sites in the H3 cells in this dataset, since 50% of the cells have less than 8 sites, and 50% of the cells have more than 8 sites.

Generate our Gi* statistic. 
Use the most typical parameters. The exception is the search radius which is always domain specific.

The output here will show, among other things, a Z score and P value. A Z score shows how many standard deviations from the mean of the neighborhood the value is and the P score tells  the chance that value is from random variation rather than an actual phenomenon.

In [30]:
from sedona.stats.hotspot_detection.getis_ord import g_local
from sedona.stats.weighting import add_binary_distance_band_column

In [31]:
gi_df = g_local(
    add_binary_distance_band_column(
        hexes_df,
        neighbor_search_radius_degrees,
        include_self=True,
    ),
    "num_places",
    "weights",
    star=True
).cache()


In [33]:
gi_df.show(4)

+------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|            h3Cell|num_places|            geometry|             weights|                   G|                  EG|                  VG|                  Z|                  P|
+------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|613207893889515519|        12|POLYGON ((-122.10...|[{{61320789397549...|0.014833640637198718|0.013219284603421462|5.542296862370928E-5|0.21684750476435208|0.41416359460836905|
|613207893973401599|       118|POLYGON ((-122.13...|[{{61320789782377...|0.026909851995814024|0.013219284603421462|5.542296862370928E-5| 1.8389780914079021|0.03295920426865351|
|613207898505347071|         2|POLYGON ((-121.98...|[{{61320789844243...|0.001777379113303...|0.006998444790046656|

In [34]:
gi_df.drop("weights", "h3Cell", "geometry").orderBy(f.col("P").asc()).show()

+----------+-------------------+--------------------+--------------------+------------------+--------------------+
|num_places|                  G|                  EG|                  VG|                 Z|                   P|
+----------+-------------------+--------------------+--------------------+------------------+--------------------+
|       250|0.08754007408514809|0.013219284603421462|5.542296862370928E-5|  9.98310001884801|                 0.0|
|       908|0.16097739240211956|0.013219284603421462|5.542296862370928E-5|19.847528249317246|                 0.0|
|       218|0.11812096144582315|0.013219284603421462|5.542296862370928E-5|14.090861243071908|                 0.0|
|       781|0.12757263168385907|0.013219284603421462|5.542296862370928E-5| 15.36045175723997|                 0.0|
|       301|0.08998189398847195|0.012441679626749611|5.220389943140836E-5|10.731873012017438|                 0.0|
|        13|0.06945067357685088| 0.01088646967340591|4.575034650957284E-5|  8.65

In [35]:
from sedona.maps.SedonaKepler import SedonaKepler

kmap = SedonaKepler.create_map(places_df, "places")

SedonaKepler.add_df(
    kmap,
    gi_df.drop("weights"),
    "cells"
)

kmap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'places': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [38]:
gi_df = g_local(
    add_binary_distance_band_column(
        hexes_df,
        neighbor_search_radius_degrees,
        include_self=True,
    ),
    "num_places",
    "weights",
    star=True
).cache()

gi_df.drop("weights", "h3Cell", "geometry").orderBy(f.col("P").asc()).show()

25/03/24 16:23:07 WARN CacheManager: Asked to cache already cached data.        


+----------+-------------------+--------------------+--------------------+------------------+--------------------+
|num_places|                  G|                  EG|                  VG|                 Z|                   P|
+----------+-------------------+--------------------+--------------------+------------------+--------------------+
|       250|0.08754007408514809|0.013219284603421462|5.542296862370928E-5|  9.98310001884801|                 0.0|
|       908|0.16097739240211956|0.013219284603421462|5.542296862370928E-5|19.847528249317246|                 0.0|
|       218|0.11812096144582315|0.013219284603421462|5.542296862370928E-5|14.090861243071908|                 0.0|
|       781|0.12757263168385907|0.013219284603421462|5.542296862370928E-5| 15.36045175723997|                 0.0|
|       301|0.08998189398847195|0.012441679626749611|5.220389943140836E-5|10.731873012017438|                 0.0|
|        13|0.06945067357685088| 0.01088646967340591|4.575034650957284E-5|  8.65

In [39]:
from sedona.maps.SedonaKepler import SedonaKepler

kmap = SedonaKepler.create_map(places_df, "places")

SedonaKepler.add_df(
    kmap,
    gi_df.drop("weights"),
    "cells"
)

kmap

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'places': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [12]:
spark = SparkSession.builder \
    .appName("Polygon Example") \
    .getOrCreate()

25/03/24 15:04:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [13]:
# Współrzędne poligonu
polygon_coords = [
    (-122.380829, 47.870302),
    (-122.048492, 47.759637),
    (-121.982574, 47.531111),
    (-122.408295, 47.50978),
    (-122.44812, 47.668162),
    (-122.380829, 47.870302)  # Zamknięcie poligonu
]


data = [Row(lon=coord[0], lat=coord[1]) for coord in polygon_coords]

In [14]:

polygon_coords = [
    (-122.380829, 47.870302),
    (-122.048492, 47.759637),
    (-121.982574, 47.531111),
    (-122.408295, 47.50978),
    (-122.44812, 47.668162),
    (-122.380829, 47.870302)  # Zamknięcie poligonu
]


data = [Row(lon=coord[0], lat=coord[1]) for coord in polygon_coords]

In [16]:

polygon_df = spark.createDataFrame(data)

e
polygon_df.show()

+-----------+---------+
|        lon|      lat|
+-----------+---------+
|-122.380829|47.870302|
|-122.048492|47.759637|
|-121.982574|47.531111|
|-122.408295| 47.50978|
| -122.44812|47.668162|
|-122.380829|47.870302|
+-----------+---------+



In [17]:

polygon_df.createOrReplaceTempView("polygon_table")

In [18]:
SedonaKepler.create_map(polygon_df, "Hiking Trails")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Hiking Trails': {'index': [0, 1, 2, 3, 4, 5], 'columns': ['lon', 'lat'], 'data': [[-122.380829…